In [1]:
import os
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('float_format', '{:f}'.format)

In [2]:
theater = pd.read_csv('theaters_191128.csv')
theater.sample()

,광역단체,기초단체,영화상영관코드,영화상영관명,총 스크린수,총 좌석수,필름 상영관수,2D 상영관수,3D 상영관수,4D 상영관수,IMAX 상영관수,상설여부,특별관운영여부,가입여부,전송사업자명,개관일,영업상태,운영형태,주소,전화번호,홈페이지
36,부산시,북구,11019,CGV 화명,7,756,0,9,2,0,0,상설,nan,가입,CJ올리브네트웍스(주),2013-07-01,영업,직영,부산광역시 북구 화명동 2275-1 번지 센추리 빌딩,1544-1122,http://www.cgv.co.kr


In [3]:
theater['영화상영관코드'].dtype

dtype('int64')

In [4]:
theater['영화상영관코드'] = theater['영화상영관코드'].astype(str).apply(lambda x: ('0'+'0'+x)[-6:])
#theater['영화상영관코드']

In [5]:
code = theater[['영화상영관코드', '영화상영관명']]

In [6]:
len(code)

553

# 상영관정보 데이터 크롤링 
http://www.kobis.or.kr/kobis/business/mast/thea/findTheaterInfoList.do  

완료되었으므로 다시 돌릴 필요 없음. 슬랙에서 '상영관정보.zip'을 다운받아 이 노트북이 돌아가는 폴더에 압축 풀어주세요.

In [19]:
URL = f'http://www.kobis.or.kr/kobis/business/mast/thea/findTheaterCodeLayer.do'

In [57]:
for i, row in code.iterrows():
    theaCd = row['영화상영관코드']
    theaNm = row['영화상영관명'].strip().replace(' ', '%20').replace('(', '%28').replace(')', '%29')
    os.system(f'curl "{URL}?theaCd={theaCd}" > 상영관정보-{theaNm}.html')

# 상영내역 데이터 크롤링
http://www.kobis.or.kr/kobis/business/mast/thea/findShowHistorySc.do

In [18]:
URL = f'http://www.kobis.or.kr/kobis/business/mast/thea/findShowHistoryScXls.do'

In [27]:
for i, row in code.iterrows():
# 일부 상영관의 데이터만 가져올 때는 code[n:m].iterrows() 로 바꾸고 n과 m에 인덱스 숫자를 넣음
    theaCd = row['영화상영관코드']
    theaNm = row['영화상영관명'].strip().replace(' ', '%20').replace('(', '%28').replace(')', '%29')
    os.system(f'curl "{URL}?theaCd={theaCd}&showStartDt=2015-01-01&showEndDt=2019-10-31&choice=2&sTheaNm={theaNm}"' \
              f' > 상영내역-{theaNm}.html')

# 상영내역 데이터 + 상영관정보 데이터 merge
'상영관정보'에서 관별 좌석 수 데이터를 가져와서 '상영내역'에 붙인 후 저장.  
저장된 csv 파일은 'Theaters_merged_files' 폴더에 있음.

In [41]:
for nm in code['영화상영관명']:
# 위에서 지정해준 인덱스 숫자 n과 m을 사용해 code['영화상영관명'][n:m] 으로 변경
    theaNm = nm.strip().replace(' ', '%20').replace('(', '%28').replace(')', '%29')
    time = pd.read_html(f'상영내역-{theaNm}.html')[1]
    capacity = pd.read_html(f'상영관정보-{theaNm}.html', encoding='utf-8')[1].T.drop('스크린코드', axis=0).rename(columns={0:'상영관', 1:'좌석수'})
    time.merge(capacity, how='left', on='상영관').to_csv(f'../Theaters_merged_files/{theaNm}.csv', index=False)

### 데이터 추가: CGV 성신여대입구 
2019-08-27까지는 CGV(5관)였는데 CGV(3관), 씨네큐(2관)으로 나눠짐.  
위에서 크롤링할 때 8월 27일까지의 CGV 데이터와 씨네큐 데이터를 가져왔고, 8월 28일 이후의 CGV 데이터는 가져오지 못했으므로 따로 불러온다.

In [33]:
theaCd = '001271'
theaNm = 'CGV%20성신여대입구_new'

# 상영관정보 크롤링
os.system(f'curl "http://www.kobis.or.kr/kobis/business/mast/thea/findTheaterCodeLayer.do?theaCd={theaCd}" > 상영관정보-{theaNm}.html')

# 상영내역 크롤링
os.system(f'curl "http://www.kobis.or.kr/kobis/business/mast/thea/findShowHistoryScXls.do?theaCd={theaCd}&showStartDt=2019-08-28&showEndDt=2019-10-31&choice=2&sTheaNm={theaNm}" > 상영내역-{theaNm}.html')

# merge
time = pd.read_html(f'상영내역-{theaNm}.html')[1]
capacity = pd.read_html(f'상영관정보-{theaNm}.html', encoding='utf-8')[1].T.drop('스크린코드', axis=0).rename(columns={0:'상영관', 1:'좌석수'})
time.merge(capacity, how='left', on='상영관').to_csv(f'../Theaters_merged_files/{theaNm}.csv', index=False)

### 데이터 추가: CGV 청주(서문), CGV 용산아이파크몰, CGV 송파
상영관정보 데이터는 있으나 상영내역 크롤링 시 계속 에러난 것들  
(index: 14, 22, 134)

In [35]:
# CGV 청주(서문)

theaCd = '004017'
theaNm = 'CGV%20청주%28서문%29'

# 상영내역 크롤링
os.system(f'curl "http://www.kobis.or.kr/kobis/business/mast/thea/findShowHistoryScXls.do?theaCd={theaCd}&showStartDt=2015-01-01&showEndDt=2019-10-31&choice=2&sTheaNm={theaNm}" > 상영내역-{theaNm}.html')

# merge
time = pd.read_html(f'상영내역-{theaNm}.html')[1]
capacity = pd.read_html(f'상영관정보-{theaNm}.html', encoding='utf-8')[1].T.drop('스크린코드', axis=0).rename(columns={0:'상영관', 1:'좌석수'})
time.merge(capacity, how='left', on='상영관').to_csv(f'../Theaters_merged_files/{theaNm}.csv', index=False)

ValueError: No tables found

In [36]:
# CGV 용산아이파크몰

theaCd = '001088'
theaNm = 'CGV%20용산아이파크몰'

# 상영내역 크롤링
os.system(f'curl "http://www.kobis.or.kr/kobis/business/mast/thea/findShowHistoryScXls.do?theaCd={theaCd}&showStartDt=2015-01-01&showEndDt=2019-10-31&choice=2&sTheaNm={theaNm}" > 상영내역-{theaNm}.html')

# merge
time = pd.read_html(f'상영내역-{theaNm}.html')[1]
capacity = pd.read_html(f'상영관정보-{theaNm}.html', encoding='utf-8')[1].T.drop('스크린코드', axis=0).rename(columns={0:'상영관', 1:'좌석수'})
time.merge(capacity, how='left', on='상영관').to_csv(f'../Theaters_merged_files/{theaNm}.csv', index=False)

ValueError: No tables found

In [37]:
# CGV 송파

theaCd = '001156'
theaNm = 'CGV%20송파'

# 상영내역 크롤링
os.system(f'curl "http://www.kobis.or.kr/kobis/business/mast/thea/findShowHistoryScXls.do?theaCd={theaCd}&showStartDt=2015-01-01&showEndDt=2019-10-31&choice=2&sTheaNm={theaNm}" > 상영내역-{theaNm}.html')

# merge
time = pd.read_html(f'상영내역-{theaNm}.html')[1]
capacity = pd.read_html(f'상영관정보-{theaNm}.html', encoding='utf-8')[1].T.drop('스크린코드', axis=0).rename(columns={0:'상영관', 1:'좌석수'})
time.merge(capacity, how='left', on='상영관').to_csv(f'../Theaters_merged_files/{theaNm}.csv', index=False)

ValueError: No tables found